In [1]:
import pandas
import numpy
import random
import copy
import csv
import collections
import time
import pandas as pd

# Defining Classes.

In [2]:
class Course():
  def __init__(self):
    self.course_id = 0
    self.course_code = ""
    self.course_name = ""

class Teacher():
  def __init__(self):
    self.teacher_id = 0
    self.teacher_name = ""
                                         
class Timeslot():                                      
  def __init__(self,day,time):
    # self.timeslot_id = 0 
    self.day = day
    self.time = time
  def __hash__(self):
    return hash(self.day) ^ hash(self.time)
  def __eq__(self,obj):
    return self.day == obj.day and self.time == obj.time
    
#Chromosome Class
class Timetable():
  def __init__(self,chromosome):
    self.chromosome = chromosome
  
class Student():
  def __init__(self):
    self.student_id = 0
    self.student_name = ""

#Defining timeslots, days of the week and classrooms.
slots = ["9:00-12:00","2:00-5:00"]
week = ["Monday-1","Tuesday-1","Wednesday-1","Thursday-1","Friday-1",
        "Monday-2","Tuesday-2","Wednesday-2","Thursday-2","Friday-2"
       ] 
        # "Monday-3","Tuesday-3","Wednesday-3","Thursday-3","Friday-3"]
classrooms = ["C-301","C-302","C-303","C-304","C-305","C-306","C-307","C-308","C-309","C-310"]

#Reading datasets.

In [3]:
#Reading list of courses.
courses = []

with open("courses.csv", mode = 'r') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter = ',')
  line_count = 0
  for row in csv_reader:
    if any(obj.course_code == row[0] for obj in courses):
      continue
    course = Course()
    course.course_code = row[0]
    course.course_name = row[1]
    course.course_id = line_count
    line_count += 1
    courses.append(course)

for n in courses:
  print(n.course_id, n.course_name, n.course_code)

0 Object Oriented Programming CS217
1 Digital Logic Design EE227
2 Discrete Structures CS211
3 Intro to Software Engineering SE110
4 Programming Fundamentals CS118
5 Database Systems CS219
6 Operating Systems CS220
7 Design & Analysis of Algorithms CS302
8 Digital Forensics CY2012
9 Computer Networks CS307
10 Software Engineering CS328
11 Computer Organization and Assembly Language EE229
12 Programming for AI AI2011
13 Big Data Analytics DS3011
14 Data Structures CS218
15 Differential Equations MT224
16 Pakistan Studies SS113
17 Marketing Management MG220
18 Fundamentals of Management MG223
19 Islamic and Religious Studies SS111
20 Communication & Presentation Skills SS152
21 Psychology SS118
22 Probability and Statistics MT205


In [4]:
#Reading list of teachers.
teachers = []

with open("teachers.csv", mode = "r") as teacher_file:
  line_count = 0
  csv_reader = csv.reader(teacher_file, delimiter = '\n')
  for row in csv_reader:
    if any(obj.teacher_name == row for obj in teachers):
      continue
    temp = ""
    for i in row:
      temp += i
    if temp is not "":
      teacher = Teacher()
      teacher.teacher_id = line_count
      teacher.teacher_name = temp
      teachers.append(teacher)
      line_count += 1

for n in teachers:
  print(n.teacher_id, n.teacher_name)

0 Ayesha Bano
1 Aqeel Shahzad
2 Farah Naz
3 Hamda Khan
4 Usman Rashid
5 Farah Jabeen Awan
6 Sara Aziz
7 Gul e Aisha
8 Maimoona Rassol
9 Sajid Khan
10 Tayyab Nadeem
11 Mehboobullah
12 Muhammad bin Qasim
13 Zainab Moin
14 Sumera Abbas
15 Sadia Nauman
16 Shahzad Mehmood
17 Sanaa Ilyas
18 Nagina Safdar
19 Asma Nisa
20 Sehrish Hassan
21 Waqas Munir
22 Usman Ashraf
23 Muhammad Usman
24 Naveed Ahmad
25 Zainab Abaid
26 Rohail Gulbaz
27 Hassan Raza
28 Javaria Imtiaz
29 Maheen Arshad
30 Zohaib Iqbal
31 Mehreen Alam
32 Mehwish Hassan
33 Shams Farooq
34 Ameen Chilwan
35 Shafaq Riaz
36 Khadija Farooq
37 Amna Irum
38 Irum Inayat
39 Bilal Khalid
40 Shehreyar Rashid
41 Umair Arshad
42 Subhan Ullah
43 Ejaz Ahmed
44 Kifayat Ullah
45 Waseem Shahzad
46 Adnan Tariq
47 Asif Naeem
48 Hasan Mujtaba
49 Sidra Khalid
50 Faisal Cheema
51 Shoaib Mehboob
52 Tayyaba Zainab
53 Arshad Islam
54 Noor ul Ain
55 Zeeshan Qaiser
56 Muhammad Asim
57 Behjat Zuhaira
58 Kashif Munir
59 Hammad Majeed
60 Noreen Jamil
61 Hassan Mu

In [5]:
#Reading list of students.
students = []

with open("studentNames.csv", mode = 'r') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter = ',')
  line_count = 0
  for row in csv_reader:
    if any(obj.student_name == row[0] for obj in students):
      continue
    student = Student()
    student.student_name = row[0]
    student.student_id = line_count
    line_count += 1
    students.append(student)

for n in students:
  print(n.student_id, n.student_name)

0 Mohammed Azam
1 Mohamed F Bakaram
2 Manal Aldaihani
3 Ibrar Manawer
4 Adam Flowers
5 Lilia Kyuchukova
6 Sarina Patel
7 Aleena Unadkat
8 Ibrahim Adegboye
9 Salma Khanam
10 Kiran K Chahal
11 Omar F Khan
12 Seema Mandalia
13 Asim Ali
14 Yasmin Ahmed
15 Sheila Hughton
16 Sarah Taylor
17 Anja Sarlah
18 Syed M Abbas
19 Hanna I Siahaan
20 Sarah Austin
21 Mohammed Riyaz Shahul Hameed Mohammed Yakub
22 Halima Sayed
23 Sarah L Barber-Jones
24 Meryem Mehmetbasoglu
25 Adam L Finch
26 Sarah Hanley
27 Sarah Fogg
28 Tara Glickman
29 Reem N Hassan
30 Amir Ahmed
31 Sarah French
32 Sarah J Roberts
33 Sam D Edwards
34 Sam Duncanson
35 Amber Summerfield
36 Mika Tatsumoto
37 Maria Antonia Morua Leyva
38 Ann D Brix
39 Jina Zhao
40 Zahir Hussain
41 Syed M Ghufran
42 Adam C Ryan
43 Adam Sanders
44 Mohamed A Baalousha
45 Naseem Akram
46 Ann Tai
47 Ann White
48 Sarah Wilson
49 Judi Glen
50 Ayan Lowe
51 Sarah Montgomery
52 Meltem Naylor
53 Damian A Cummings
54 Maria A Grenfell
55 Nabila Altaf
56 Sarah Nolasco


In [6]:
#Saving list of courses registered against student.
student_registrations = dict()

with open("studentCourse.csv", mode = 'r') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter = ",")
  line_count = 0
  for row in csv_reader:
    if line_count == 0:
      line_count += 1
      continue
    else:
      if row[1] not in student_registrations: # Name of student.
        student_registrations[row[1]] = []
        student_registrations[row[1]].append(row[2])
      else:
        student_registrations[row[1]].append(row[2])

for key,value in student_registrations.items():
  print(key,value)

Sam D Edwards ['AI2011', 'SS152', 'EE229', 'CS220']
Sheila Hughton ['DS3011', 'CS328', 'CS118', 'CS302']
Yasmin Ahmed ['SE110', 'CS307', 'SS113', 'CY2012']
Sarah N Md Sallehuddin Khan ['EE229', 'CS328', 'CS307', 'MG223']
Sarah Nolasco ['AI2011', 'CS328', 'CS307', 'MG223']
Jenna Riley ['EE229', 'CS211', 'SE110', 'MT205']
Usman Rafiq ['CS307', 'CS218', 'CS118', 'CS302']
Reem N Hassan ['MG220', 'SS118', 'SS152', 'MT224']
Sarah Hinett ['CS328', 'CS217', 'CS307', 'SS152']
Kamal Anwar ['EE229', 'MG220', 'SS152', 'MT205']
Mika Tatsumoto ['CS219', 'CS217', 'CS118', 'CS218']
Muhammad Ijaz-Ul-Haq ['AI2011', 'CS211', 'SS113', 'EE229']
Abdul Gafur ['SS118', 'SE110', 'DS3011', 'CS217', 'CS328']
Ana Vukojevic ['CS307', 'AI2011', 'SS152', 'SS113']
Arooba Zahoor ['CS302', 'SS113', 'AI2011', 'MT205']
Ahmad F Yang Abd Talib ['MG223', 'CS307', 'CS211', 'CS218']
Natasha Leeson ['CS328', 'SS113', 'CS307', 'CS211', 'SS152']
Ramesh R Singh ['MG220', 'MT224', 'MG223', 'CS220']
Sara Zamberlan ['CS211', 'CS217'

Generating population randomly.

In [7]:
#Generating a population of size 'popSize' randomly.
def generatePopulation(crs, tch, popSize):  
    population = []
    num_exams = len(crs)
    for j in range(popSize):
      courses = []
      teachers = []
      timeslots = []
      courses = copy.deepcopy(crs)
      teachers = copy.deepcopy(tch)
      timeslots = slots
      days = week
      chromosome = []
      for i in range(num_exams):
        course = random.choice(courses)
        teacher = random.choice(teachers)
        day = random.choice(days)
        slot = random.choice(slots)
        classroom = random.choice(classrooms)
        chromosome.append(course.course_code)
        chromosome.append(teacher.teacher_name)
        chromosome.append(day)
        chromosome.append(slot)
        chromosome.append(classroom)
        courses.remove(course)
        teachers.remove(teacher)
      population.append(chromosome)  
    return population

# Hard Constraints

Finding the papers which have been set at a specific time and date.

In [8]:
def getPaperClashes(timetable):
    index = 0
    paper_clashes = dict()
    for data in timetable:
      if index == 0: #Course Code.
        course_code = data
      elif index == 1: #Teacher Name.
        teacher_name = data
      elif index == 2: #Day.
        day = data
      elif index == 3: #Time slot.
        time_slot = data
      elif index == 4: #Room.
        room = data
        # print("Current Line : ", course_code, teacher_name, day, time_slot, room)
        timeslot = Timeslot(day,time_slot)
        if timeslot not in paper_clashes.keys():
          paper_clashes[timeslot] = []
          paper_clashes[timeslot].append(course_code)
          # print("Adding new entry :" , (time_slot,day), course_code)
        else:
          paper_clashes[timeslot].append(course_code)
          # print("Adding to existing.", (time_slot,day), course_code)
        index = 0
        continue
      index = index + 1
    return paper_clashes

#Testing function.    
testpopulation = generatePopulation(courses,teachers,20)
test_paper_clashes = getPaperClashes(testpopulation[0])
for key,value in test_paper_clashes.items():
  print(key.day,key.time,value)

Friday-1 9:00-12:00 ['EE227', 'CS220', 'CS217']
Tuesday-1 9:00-12:00 ['MG223']
Monday-1 9:00-12:00 ['MT224', 'SE110']
Friday-1 2:00-5:00 ['CS302', 'AI2011', 'MG220', 'CY2012']
Thursday-1 2:00-5:00 ['EE229', 'CS211', 'CS118']
Thursday-2 2:00-5:00 ['SS113', 'CS219', 'CS307']
Tuesday-1 2:00-5:00 ['SS118', 'CS328']
Friday-2 2:00-5:00 ['CS218']
Wednesday-1 9:00-12:00 ['MT205']
Monday-1 2:00-5:00 ['SS111']
Monday-2 9:00-12:00 ['DS3011']
Wednesday-2 2:00-5:00 ['SS152']


Calculating the number of students from a timetable which have clashing papers and returning the count.

In [9]:
#Utility function to find intersection of two lists.
def intersection(list1, list2):
    intersecting_values = [value for value in list1 if value in list2]
    return intersecting_values

def calculateClashes(paper_clashes,student_registrations):
  numClashes = 0
  students_list = dict()
  for key,value in paper_clashes.items():
    if len(value) >= 2: # Atleast two subjects in the same slot.
      for course in value:
        students_list[course] = []
        for student_name,subjects in student_registrations.items():
          if course in subjects:
            students_list[course].append(student_name)

  for key,value in paper_clashes.items():
    if len(value) >= 2:
      for i in range(len(value)):
        list1 = students_list[value[i]]
        for j in range(i+1,len(value)):
          list2 = students_list[value[j]]
          if len(intersection(list1,list2)) > 0:
            # print("Clashes between ", value[i], "and ", value[j] ," are : ", len(intersection(list1,list2)) )
            numClashes = numClashes + len(intersection(list1,list2))
            # print("Clashes of ", value[i], " and " , value[j], " : ", intersection(list1,list2))
  # print("Total clashes found of : ", numClashes , " students.")
  return numClashes
#Testing function.
calculateClashes(test_paper_clashes,student_registrations)

78

Finding those teachers which have been assigned to specific day and time.

In [10]:
#Find teachers who are invigilating multiple exams at the same day and time.
def getTeacherClashes(timetable):
  index = 0
  teacher_clashes = dict()
  for data in timetable:
    if index == 0: #Course Code.
      course_code = data
    elif index == 1: #Teacher Name.
      teacher_name = data
    elif index == 2: #Day.
      day = data
    elif index == 3: #Time slot.
      time_slot = data
    elif index == 4: #Room.
      room = data
      # print("Current Line : ", course_code, teacher_name, day, time_slot, room)
      timeslot = Timeslot(day,time_slot)
      if timeslot not in teacher_clashes.keys():
        teacher_clashes[timeslot] = []
        teacher_clashes[timeslot].append(teacher_name)
        # print("Adding new entry :" , (time_slot,day), teacher_name)
      else:
        teacher_clashes[timeslot].append(teacher_name)
        # print("Adding to existing.", (time_slot,day), teacher_name)
      index = 0
      continue
    index = index + 1
  return teacher_clashes

#Testing function.
test_teacher_clashes = getTeacherClashes(testpopulation[0])
for key,value in test_teacher_clashes.items():
  print(key.day,key.time,value)

Friday-1 9:00-12:00 ['Muhammad Asim', 'Amna Irum', 'Shams Farooq']
Tuesday-1 9:00-12:00 ['Nagina Safdar']
Monday-1 9:00-12:00 ['Zainab Abaid', 'Zeeshan Qaiser']
Friday-1 2:00-5:00 ['Sidra Khalid', 'Farah Jabeen Awan', 'Asma Nisa', 'Noreen Jamil']
Thursday-1 2:00-5:00 ['Sehrish Hassan', 'Mehwish Hassan', 'Irum Inayat']
Thursday-2 2:00-5:00 ['Usman Rashid', 'Sara Aziz', 'Shafaq Riaz']
Tuesday-1 2:00-5:00 ['Zainab Moin', 'Hammad Majeed']
Friday-2 2:00-5:00 ['Rohail Gulbaz']
Wednesday-1 9:00-12:00 ['Sanaa Ilyas']
Monday-1 2:00-5:00 ['Farah Naz']
Monday-2 9:00-12:00 ['Shahzad Mehmood']
Wednesday-2 2:00-5:00 ['Sumera Abbas']


Returns number of teachers which are invigilating more than one course at the same time.

In [11]:
def calculateTeacherClashes(teacher_clashes):
  numClashes = 0
  for key,value in teacher_clashes.items():
    if len(value) >= 2:
      numClashes = numClashes + len([item for item, count in collections.Counter(value).items() if count > 1])
  # print("Total clashes found of : ", numClashes , " teachers.")
  return numClashes
#Testing Function.
calculateTeacherClashes(test_teacher_clashes)

0

Finds the number of teachers who are invigilating back to back (consecutive) exams on the same day.

In [12]:
#Find teachers who are invigilating consecutive exams.
def getConsecutiveTeacherClashes(teacher_clashes):
  numClashes = 0
  for day1 in teacher_clashes.keys():
    checkDays = False
    for day2 in teacher_clashes.keys():
      if day1 == day2:
        checkDays = True
        continue
      if checkDays == True:
        #Check if day same and timeslot different and teacher repeating.
        if day1.day == day2.day:
          # print("Checking days : ", day1.day, day1.time, " and ", day2.day, day2.time)
          list1 = teacher_clashes[day1]
          list2 = teacher_clashes[day2]
          if len(intersection(list1,list2)) > 0:
            # print("Teacher clashes found for day: ", day1.day, " Timings : ", day1.time, " " ,day2.time)
            # print(list1)
            # print(list2)
            numClashes = numClashes + len(intersection(list1,list2))
          # else:
          #   print("No teacher clashes found!")
  return numClashes
                      

getConsecutiveTeacherClashes(test_teacher_clashes)

0

Find the classrooms which have been assigned to courses at a specific day and time.

In [13]:
def getClassroomClashes(timetable):
  index = 0
  classroom_clashes = dict()
  for data in timetable:
    if index == 0: #Course Code.
      course_code = data
    elif index == 1: #Teacher Name.
      teacher_name = data
    elif index == 2: #Day.
      day = data
    elif index == 3: #Time slot.
      time_slot = data
    elif index == 4: #Room.
      room = data
      # print("Current Line : ", course_code, teacher_name, day, time_slot, room)
      timeslot = Timeslot(day,time_slot)
      if timeslot not in classroom_clashes.keys():
        classroom_clashes[timeslot] = []
        classroom_clashes[timeslot].append(room)
        # print("Adding new entry :" , (time_slot,day), teacher_name)
      else:
        classroom_clashes[timeslot].append(room)
        # print("Adding to existing.", (time_slot,day), teacher_name)
      index = 0
      continue
    index = index + 1
  return classroom_clashes
#Testing function.
test_classroom_clashes = getClassroomClashes(testpopulation[0])
for key,value in test_classroom_clashes.items():
  print(key.day,key.time,value)

Friday-1 9:00-12:00 ['C-302', 'C-301', 'C-301']
Tuesday-1 9:00-12:00 ['C-309']
Monday-1 9:00-12:00 ['C-301', 'C-310']
Friday-1 2:00-5:00 ['C-305', 'C-303', 'C-310', 'C-307']
Thursday-1 2:00-5:00 ['C-303', 'C-309', 'C-305']
Thursday-2 2:00-5:00 ['C-303', 'C-302', 'C-309']
Tuesday-1 2:00-5:00 ['C-308', 'C-302']
Friday-2 2:00-5:00 ['C-304']
Wednesday-1 9:00-12:00 ['C-302']
Monday-1 2:00-5:00 ['C-304']
Monday-2 9:00-12:00 ['C-301']
Wednesday-2 2:00-5:00 ['C-308']


Find the number of classrooms which have been assigned to multiple courses at the same day and time, i.e. one classroom has been assigned to two or more courses at the same day and time.

In [14]:
def calculateClassroomClashes(classroom_clashes):
  numClashes = 0
  for key,value in classroom_clashes.items():
    if len(value) >= 2:
      numClashes = numClashes + len([item for item, count in collections.Counter(value).items() if count > 1])
  # print("Total clashes found of : ", numClashes , " classrooms.")
  return numClashes
#Testing function.
calculateClassroomClashes(test_classroom_clashes)

1

# Soft Constraints

Find the number of students appearing in consecutive papers on the same day.

In [15]:
def calculateConsecutivePaperClashes(paper_clashes,student_registrations):
  numClashes = 0
  students_list = dict()
  daysChecked = []
  for key,value in paper_clashes.items():
    for course in value:
      students_list[course] = []
      for student_name, subjects in student_registrations.items():
        if course in subjects:
          students_list[course].append(student_name)

  
  for key1,value1 in paper_clashes.items():
    for key2,value2 in paper_clashes.items():
      if key1.day == key2.day: #Same day.
        if key1.time != key2.time: #Different times.
          if key1.day not in daysChecked:
            daysChecked.append(key1.day)
            for i in range(len(value1)):
              list1 = students_list[value1[i]]
              for j in range(len(value2)):
                list2 = students_list[value2[j]]
                if len(intersection(list1,list2)) > 0:
                  # print("Consecutive clashes between", value1[i], " and ", value2[j], " are: ", len(intersection(list1,list2)))
                  numClashes = numClashes + len(intersection(list1,list2))
                # print("Consecutive Clashes of ", value1[i], " and " , value2[j], " : ", intersection(list1,list2))
  # print("Total consecutive clashes found of : ", numClashes , " students.")
  return numClashes
#Testing function.
calculateConsecutivePaperClashes(test_paper_clashes,student_registrations)

66

Finds the number of students who are appearing in both CS and Management courses, and have their CS course scheduled before their management course.

In [16]:
def calculateManagementCScourseClashes(paper_clashes,student_registrations):
  numClashes = 0
  students_list = dict()
  sorted_paper_clashes = dict()
  daysChecked = []
  for key,value in paper_clashes.items():
    for course in value:
      students_list[course] = []
      for student_name, subjects in student_registrations.items():
        if course in subjects:
          students_list[course].append(student_name)

  for day in week:
    for slot in slots:
      for key,value in paper_clashes.items():
        if key.day == day and key.time == slot:
          sorted_paper_clashes[key] = value

  # for key,value in sorted_paper_clashes.items():
    # print(key.day,key.time,value)
    
  for key,value in sorted_paper_clashes.items():
    for course in value:
      if course[0] == 'C' and course[1] == 'S':#Choose a CS course.
        csCourse = course
        # print("Checking if any Management course after ", csCourse)
        checkDays = False
        for key2,value2 in sorted_paper_clashes.items():
          if key == key2:
            checkDays = True
            continue
          if checkDays == True:
            for course2 in value2:
              if course2[0] == 'M' and course2[1] =='G':#Found an MG course.
                mgCourse = course2
                list1 = students_list[csCourse]
                list2 = students_list[mgCourse]
                # print("Management course found! Checking courses ", csCourse , " and ", mgCourse)
                # print(print("Students taking both of these courses are : ", len(intersection(list1,list2))))
                if len(intersection(list1,list2)) > 0:
                  # print("CS Course ", csCourse , " was scheduled before MG Course ", mgCourse, " and " , len(intersection(list1,list2)) ," clashes were found!!!!!")
                  numClashes += len(intersection(list1,list2))
  return numClashes
  
calculateManagementCScourseClashes(test_paper_clashes,student_registrations)

36

Implements crossover between two parents and returns new child.

In [17]:
def crossover(p_1, p_2):
  child = []
  random_1 = [*range(0,23,1)]
  random_2 = [*range(0,23,1)]
  for i in range(23):
    index = random.choice(random_1)
    random_1.remove(index)
    index = index * 5
    exam = p_1[index:index+5]
    if exam[0] not in child:
      child.extend(exam)

    if len(child) == 115:
      break
    
    index = random.choice(random_2)
    random_2.remove(index)
    index = index * 5
    exam = p_2[index:index+5]
    if exam[0] not in child:
      child.extend(exam)

  # assert(len(child)==115)
  return child

Implements mutation on child with a probability, and returns newly mutated child.

In [18]:
def mutation(child, teachers, courses, slots, week, classrooms):
  mychild = copy.deepcopy(child)
  prob = [*range(0,101,1)]
  for count, val in enumerate(child):
    chance = random.choice(prob)
    if chance <= 4:
      n = count%5
      if n == 0:
        continue
      elif n == 1:
        teacher = random.choice(teachers)
        mychild[count] = teacher.teacher_name
      elif n == 2:
        day = random.choice(week)
        mychild[count] = day
      elif n == 3:
        slot = random.choice(slots)
        mychild[count] = slot
      elif n == 4:
        room = random.choice(classrooms)
        mychild[count] = room
  return mychild


Finds fitness of a timetable depending on hard and soft constraints.

In [19]:
def calculateFitness(timetable):
  fitness = 0

  paper_clashes = dict()
  teacher_clashes = dict()
  classroom_clashes = dict()
  paper_clashes = getPaperClashes(timetable)
  teacher_clashes = getTeacherClashes(timetable)
  classroom_clashes = getClassroomClashes(timetable)

  studentsClashes = 0
  teachersClashes = 0
  consecutiveTeachersClashes = 0
  roomClashes = 0
  studentsClashes = calculateClashes(paper_clashes,student_registrations)
  teachersClashes = calculateTeacherClashes(teacher_clashes)
  consecutiveTeachersClashes = getConsecutiveTeacherClashes(teacher_clashes)
  roomClashes = calculateClassroomClashes(classroom_clashes)

   
  softClashes = calculateConsecutivePaperClashes(paper_clashes,student_registrations) + calculateManagementCScourseClashes(paper_clashes,student_registrations)
  hardClashes = studentsClashes + teachersClashes + consecutiveTeachersClashes + roomClashes
  clashes = hardClashes  + (softClashes / 10)
  if clashes == 0.0:
    print("All constraints (soft + hard) met!")
    fitness = 10000000
    return fitness
  if hardClashes == 0:
    print("All hard constraints met! Returning timetable...")
    print("Remaining soft clashes : ", softClashes)
    fitness = 10000000
    return fitness 
  fitness = (1000 / (clashes*clashes)) 
  return fitness

def getParentByRelativeFitness(matingPool):
  return random.choices(list(matingPool.keys()),weights=list(matingPool.values()),k=2)


Genetic Algorithm implementation.

In [22]:
population = []
def geneticAlgorithm(population):
  #Initialization.
  populationSize = 100
  population = generatePopulation(courses,teachers,populationSize)
  iteration = 0
  #Start of Algorithm.
  while True:
    #Evaluating fitness of each element of population.
    matingPool = dict()
    for timetable in population:
      fitness = calculateFitness(timetable)
      if (fitness == 10000000):
        # print("Fitness == ",fitness)
        return timetable
      matingPool[tuple(timetable)] = fitness
    print("Iteration ",iteration, ": [", end=" ")
    sortedPool = {k: v for k,v in sorted(matingPool.items(),key=lambda item: item[1],reverse=True)}
    for key,value in sortedPool.items():
      print(value, end=", ")
    print(" ]", end= " ")
    print()
    iteration = iteration + 1
    #Reproduction phase.
    newPopulation = []
    for i in range(populationSize):
      #Selecting two parents based on relative fitness.
      parents = getParentByRelativeFitness(matingPool)
      crossoverChild = crossover(list(parents[0]),list(parents[1]))
      mutatedChild = mutation(crossoverChild, teachers, courses, slots, week, classrooms)
      newPopulation.append(mutatedChild)
    population = newPopulation
idealTimetable = geneticAlgorithm(population)


Iteration  0 : [ 0.43766740778347724, 0.4340277777777778, 0.41649312786339027, 0.40647762747138394, 0.38750077500155, 0.3741268813905547, 0.3684041835979089, 0.35870064279155195, 0.3467779130211638, 0.3354398958794563, 0.31548826541396796, 0.31105263321606647, 0.3067098923141568, 0.2993259180325906, 0.29522561141224124, 0.29320698067179585, 0.29320698067179585, 0.2922054204105486, 0.2912089832147142, 0.289231338794021, 0.289231338794021, 0.2796389302133086, 0.2678659920015215, 0.2678659920015215, 0.26525480376449617, 0.2593085278795563, 0.2543694308992722, 0.2527543909756572, 0.248000496000992, 0.248000496000992, 0.24337946996819032, 0.2381496723060509, 0.23668639053254437, 0.2352365538785803, 0.22410269281795692, 0.2227667631989307, 0.22210327358014936, 0.21882987290360986, 0.2124964938078522, 0.2082253164504247, 0.20234354291403034, 0.2011971228811428, 0.20006041824631035, 0.19893331954062315, 0.1972604469132685, 0.19615689413020107, 0.19560858721697882, 0.19397738999542213, 0.192901

In [25]:
def printSolution(timetable):
  fitness = 0

  paper_clashes = dict()
  teacher_clashes = dict()
  classroom_clashes = dict()
  paper_clashes = getPaperClashes(timetable)
  teacher_clashes = getTeacherClashes(timetable)
  classroom_clashes = getClassroomClashes(timetable)

  studentsClashes = 0
  teachersClashes = 0
  consecutiveTeachersClashes = 0
  roomClashes = 0
  studentsClashes = calculateClashes(paper_clashes,student_registrations)
  teachersClashes = calculateTeacherClashes(teacher_clashes)
  consecutiveTeachersClashes = getConsecutiveTeacherClashes(teacher_clashes)
  roomClashes = calculateClassroomClashes(classroom_clashes)

    
  softClashes = calculateConsecutivePaperClashes(paper_clashes,student_registrations) + calculateManagementCScourseClashes(paper_clashes,student_registrations)
  hardClashes = studentsClashes + teachersClashes + consecutiveTeachersClashes + roomClashes
  clashes = hardClashes  + (softClashes / 10)
  print("Hard Constraints Satisfied :")
  print("->Exam scheduled for each course and each student is enrolled in at least 3 courses.")
  if studentsClashes == 0:
    print("->All students clashes resolved.")
  if teachersClashes == 0:
    print("->All teachers clashes resolved.")
  if consecutiveTeachersClashes == 0:
    print("->No teacher invigilates consecutive papers.")
  print("->All papers held between 9am and 5pm. Each exam is invigilated by a teacher. Exam is not held on weekend.")

  print()
  print("Following soft constraints have been optimized as much as possible:")
  if calculateConsecutivePaperClashes(paper_clashes,student_registrations) == 0:
    print("->No consecutive papers scheduled for any student.")
  else:
    print("->Student appearing in more than 1 exam consecutively.")
  if calculateManagementCScourseClashes(paper_clashes,student_registrations) == 0:
    print("->No management course is scheduled before CS course for a student who takes both.")
  else:
    print("->Management course is scheduled before CS course for a student who is enrolled in both.")

printSolution(idealTimetable)

Hard Constraints Satisfied :
->Exam scheduled for each course and each student is enrolled in at least 3 courses.
->All students clashes resolved.
->All teachers clashes resolved.
->No teacher invigilates consecutive papers.
->All papers held between 9am and 5pm. Each exam is invigilated by a teacher. Exam is not held on weekend.

Following soft constraints have been optimized as much as possible:
->Student appearing in more than 1 exam consecutively.
->Management course is scheduled before CS course for a student who is enrolled in both.


In [26]:
courses_assigned = []
teachers_assigned = []
days_assigned = []
time_slots_assigned = []
rooms_assigned = []
index = 0 
for data in idealTimetable:
  if index == 0: #Course Code.
    courses_assigned.append(data)
  elif index == 1: #Teacher Name.
    teachers_assigned.append(data)
  elif index == 2: #Day.
    days_assigned.append(data)
  elif index == 3: #Time slot.
    time_slots_assigned.append(data)
  elif index == 4: #Room.
    rooms_assigned.append(data)
    index = 0
    continue
  index = index + 1
schedule = {'Course': courses_assigned,
            'Teacher': teachers_assigned,
            'Day': days_assigned,
            'Time-Slot': time_slots_assigned,
            'Venue':rooms_assigned,
            }
solution = pd.DataFrame(schedule)
print(solution)

    Course             Teacher          Day   Time-Slot  Venue
0    CS328    Shehreyar Rashid  Wednesday-2  9:00-12:00  C-310
1    EE229       Hasan Mujtaba     Friday-2   2:00-5:00  C-306
2    MG223      Behjat Zuhaira     Monday-1  9:00-12:00  C-304
3    CS219        Usman Rashid     Monday-1   2:00-5:00  C-304
4    MG220      Mehwish Hassan    Tuesday-1   2:00-5:00  C-308
5    CS211        Noreen Jamil   Thursday-1  9:00-12:00  C-308
6    CS302        Usman Rashid     Monday-2  9:00-12:00  C-307
7    EE227  Muhammad bin Qasim     Friday-1  9:00-12:00  C-308
8    CS118         Irum Inayat   Thursday-1   2:00-5:00  C-309
9    MT205        Arshad Islam    Tuesday-2  9:00-12:00  C-305
10   CS218           Amna Irum  Wednesday-1  9:00-12:00  C-303
11   SE110           Asma Nisa  Wednesday-1   2:00-5:00  C-307
12  AI2011  Muhammad bin Qasim    Tuesday-2   2:00-5:00  C-301
13   MT224         Gul e Aisha     Friday-2  9:00-12:00  C-310
14   CS220      Hassan Mustafa     Monday-2  9:00-12:00